In [1]:
# !pip install torch

In [2]:
# !pip install typing_extensions==4.0.0

In [3]:
# !pip3 install Cython

In [1]:
import numpy as np
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json, udf
from pyspark.sql.types import StructType, StructField, FloatType, ArrayType, IntegerType, DoubleType
from pyspark.sql.functions import col, udf
import numpy as np
from rl_agent.Agent import Agent

# Spark session & context
spark = (SparkSession
         .builder
         .master('local')
         .appName('kafka-streaming')
         .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.11:2.4.5")
         .getOrCreate())
sc = spark.sparkContext

# Khởi tạo Kafka consumer
kafka_bootstrap_servers = 'kafka1:19091,kafka2:19092, kafka3:19093'  # Địa chỉ Kafka của các container Kafka
kafka_topic = 'nguyentri'  # Tên topic Kafka

ADABUSD=0
ETHBUSD=1
BNBBUSD=2
BTCBUSD=3
DOTBUSD=4
agent = None
closes = []
buys = []
sells = []
capital = 100000


In [2]:
from pyspark.sql.functions import from_json, udf, lit


In [3]:
# Subscribe to 1 topic
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
  .option("subscribe", kafka_topic) \
  .load()

# Tạo một action prediction UDF
def predict_action(rowData):
    global agent, closes, buys, sells, capital
    # Các bước để predict action dựa trên RL model của bạn
    # ADABUSD=0, ETHBUSD=1, BNBBUSD=2, BTCBUSD=3, DOTBUSD=4
    ADABUSD = 0
    ETHBUSD = 1
    BNBBUSD = 2
    BTCBUSD = 3
    DOTBUSD = 4
    if agent is None:
        agent = Agent(250)

#     agent = Agent(250)
#     closes = []
#     buys = []
#     sells = []
#     capital = 100000
    value = np.array(eval(rowData))
    state = value.reshape((1, 250))

    close_price = state[0][10*5*4+5*BTCBUSD]
    closes.append(close_price)
    print("len(closes) in:" , len(closes))

    action = agent.act(state)
    print("*****************************")
    print("action: ", action)
    print("close_price: ", close_price)
    print("len(agent.inventory): ", len(agent.inventory))
    if action == 1:  # buy
        if capital > close_price:
            agent.inventory.append(state)
            print("agent.inventory.append(state) SUCCESS ")
            print("len(agent.inventory): ", len(agent.inventory))
            buys.append(close_price)
            sells.append(None)
            capital -= close_price
        else:
            buys.append(None)
            sells.append(None)

    elif action == 2:  # sell
        if len(agent.inventory) > 0:
            bought_price = agent.inventory.pop(0)
            print("bought_price: ", bought_price)

            profit = close_price - bought_price[0][10*5*4+5*BTCBUSD]
            reward = max(profit, 0)
            sells.append(close_price)
            buys.append(None)
            capital += profit
        else:
            print("No bought_price: ")
            buys.append(None)
            sells.append(None)
    elif action == 0:
        buys.append(None)
        sells.append(None)
    
    return action

print("len(closes) out:" , len(closes))
# Chuyển đổi cột value thành decoded_value (UTF-8 decoding)
decoded_df = df.withColumn("decoded_value", df["value"].cast("string").alias("decoded_value"))\
    .withColumn("close_price", lit( closes.pop(0) if len(closes) > 0 else None))\
    .withColumn("buy", lit(closes.pop(0) if len(closes) > 0 else None))\
    .withColumn("sell", lit(closes.pop(0) if len(closes) > 0 else None))\

# Đăng ký UDF dự đoán action với Spark
predict_action_udf = udf(predict_action, IntegerType())

# Áp dụng UDF để dự đoán action
predicted_df = decoded_df.withColumn("action", predict_action_udf(decoded_df["decoded_value"]))

show_df = predicted_df.select("decoded_value", "action")

# In dữ liệu ra màn hình console
query2 = show_df.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

query2.awaitTermination()

len(closes) out: 0


KeyboardInterrupt: 

In [1]:
# schema = ArrayType(
#     ArrayType(
#         StructType([
#             StructField("col1", FloatType(), nullable=False),
#             StructField("col2", FloatType(), nullable=False),
#             StructField("col3", FloatType(), nullable=False),
#             StructField("col4", FloatType(), nullable=False),
#             StructField("col5", IntegerType(), nullable=False)
#         ])
#     )
# )

# # Chuyển đổi cột value thành decoded_value (UTF-8 decoding)
# decoded_df = df.withColumn("decoded_value", df["value"].cast("string").alias("decoded_value"))

# # Hàm chuyển đổi dữ liệu
# def transform_data(rowData):
#     value = np.array(eval(rowData))
#     value = value.reshape((1, 250))
#     print("data: ", value)
#     print("close: ", value[0][0])
#     print(value.shape)
#     return value.tolist()


# # Đăng ký UDF với Spark
# transform_data_udf = udf(transform_data, ArrayType(FloatType()))

# # Áp dụng UDF để chuyển đổi cột decoded_value thành giá trị
# decoded_df = decoded_df.withColumn("value", transform_data_udf(decoded_df["decoded_value"]))

# # Chọn cột value
# decoded_df = decoded_df.select("value")

# # In dữ liệu ra màn hình console
# query = decoded_df.writeStream \
#     .outputMode("append") \
#     .format("console") \
#     .start()

# # Chờ cho quá trình stream kết thúc
# query.awaitTermination()